# Community Indicators

This notebook documents the process of analyzing community indicators based on census data. Below is an outline of the workflow:

## Workflow Steps

1. **Load Dataset**  
    - Import the dataset containing barangay-level census data.

2. **Filter Relevant Provinces**  
    - Focus on the 17 provinces of interest:
      - 801: CITY OF CALOOCAN  
      - 802: CITY OF LAS PIÑAS  
      - 803: CITY OF MAKATI  
      - 804: CITY OF MALABON  
      - 805: SANTA CRUZ  
      - 806: SAMPALOC  
      - 807: CITY OF MARIKINA  
      - 808: CITY OF MUNTINLUPA  
      - 809: CITY OF NAVOTAS  
      - 810: CITY OF PARAÑAQUE  
      - 811: PASAY CITY  
      - 812: CITY OF PASIG  
      - 813: QUEZON CITY  
      - 814: CITY OF SAN JUAN  
      - 815: CITY OF TAGUIG  
      - 816: CITY OF VALENZUELA  
      - 817: PATEROS  

3. **Data Cleaning and Transformation**  
    - Coerce distance fields to numeric values, handle missing data, and apply caps for distances exceeding 30 km.  
    - Create 16 community indicators based on the dataset.

4. **Province-Level Aggregation**  
    - Compute province-level averages for the community indicators.  
    - Save the aggregated data for further analysis or merging.

5. **Visualization and Analysis**  
    - Analyze trends and patterns in the community indicators across provinces.

## Key Variables

- **`df_barangay`**: Original dataset containing barangay-level census data.  
- **`df_brg`**: Filtered and transformed dataset focused on Region 13 and the selected provinces.  
- **`distance_vars`**: List of distance-related fields capped at 30 km.  
- **`pmt16_cols`**: List of 16 community indicators created from the dataset.  
- **`prov_comm_means`**: Province-level averages of the community indicators.

This documentation serves as a guide to understanding the steps and variables used in the analysis of community indicators.

In [2]:
import pandas as pd
import numpy as np

In [3]:
DATA_PATH = '/Users/Ruhama/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Master thesis'

In [4]:
df_brg = pd.read_csv(DATA_PATH + '/Census/PHL PSA CPH 2020 PUF/PUF for CPH Form 5 (Barangay Schedule Questionnaire)/Philippines/CPH PUF 2020 Philippines-Barangay Schedule.csv')

# Display the first few rows of the DataFrame
df_brg.head()

df_brg = df_brg[df_brg['REG'] == 13]

# Display the first few rows of the filtered DataFrame
# df_brg

/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_78918/1165258429.py:1: DtypeWarning: Columns (4,5,6,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df_brg = pd.read_csv(DATA_PATH + '/Census/PHL PSA CPH 2020 PUF/PUF for CPH Form 5 (Barangay Schedule Questionnaire)/Philippines/CPH PUF 2020 Philippines-Barangay Schedule.csv')


In [5]:
df_brg.columns = df_brg.columns.str.strip()

In [5]:
# ------------------------------------------------------------------
# 1.  Build a list that has BOTH distance vars and their base flags
# ------------------------------------------------------------------
distance_cols = [
    "Q3_DISTANCE",  "Q4D_DISTANCE", "Q4A_DISTANCE", "Q4E_DISTANCE",
    "Q4F_DISTANCE", "Q4G_DISTANCE", "Q4K_DISTANCE", "Q4J_DISTANCE",
    "Q4C_DISTANCE", "Q4M_DISTANCE", "Q4I_DISTANCE", "Q4N_DISTANCE",
    "Q4P_DISTANCE", "Q4O_DISTANCE", "Q4L_DISTANCE"
]

# Add the two “plain” binary vars that don’t have _DISTANCE versions
plain_indicator_cols = ["Q1B", "Q2", "Q4Q"]

# Create the matching base-name list (e.g. Q4O)
base_cols = [c.replace("_DISTANCE", "") for c in distance_cols
             if c.replace("_DISTANCE", "") in df_brg.columns]

orig_cols = plain_indicator_cols + distance_cols + base_cols
# Optional: sort so output is ordered nicely
orig_cols = sorted(set(orig_cols))

# ------------------------------------------------------------------
# 2.  Count missing values
# ------------------------------------------------------------------
nan_table = (df_brg[orig_cols]
               .isna()
               .sum()
               .rename("num_nans")
               .reset_index()
               .rename(columns={"index": "variable"})
               .sort_values("num_nans", ascending=False))

# 3.  Preview
display(nan_table.style.format({"num_nans": "{:,}"}))


,variable,num_nans
0,Q1B,0
17,Q4I_DISTANCE,0
31,Q4P_DISTANCE,0
30,Q4P,0
29,Q4O_DISTANCE,0
28,Q4O,0
27,Q4N_DISTANCE,0
26,Q4N,0
25,Q4M_DISTANCE,0
24,Q4M,0


In [6]:
for col in orig_cols:
    print(f"Value counts for column: {col}")
    print(df_brg[col].value_counts())
    print("-" * 40)

Value counts for column: Q1B
Q1B
1    1081
2     629
Name: count, dtype: int64
----------------------------------------
Value counts for column: Q2
Q2
1    1710
Name: count, dtype: int64
----------------------------------------
Value counts for column: Q3
Q3
1    1710
Name: count, dtype: int64
----------------------------------------
Value counts for column: Q3_DISTANCE
Q3_DISTANCE
1    1617
2      93
Name: count, dtype: int64
----------------------------------------
Value counts for column: Q4A
Q4A
2    1692
1      18
Name: count, dtype: int64
----------------------------------------
Value counts for column: Q4A_DISTANCE
Q4A_DISTANCE
2    1146
1     546
       18
Name: count, dtype: int64
----------------------------------------
Value counts for column: Q4C
Q4C
2    1364
1     346
Name: count, dtype: int64
----------------------------------------
Value counts for column: Q4C_DISTANCE
Q4C_DISTANCE
1    1082
      346
2     282
Name: count, dtype: int64
---------------------------------

In [7]:
# # -----------------------------------------------------------
# # 3.  Create the community indicators
# # -----------------------------------------------------------
df_brg["poblacion"]        = (df_brg["Q1B"] == 1).astype("uint8")
df_brg["street_pattern"]   = (df_brg["Q2"]  == 1).astype("uint8")
df_brg["acc_nat_hwy"]      = (df_brg["Q3_DISTANCE"]  == 1).astype("uint8")


df_brg["cemetery"]      = (df_brg["Q4D"] == 1).astype("uint8")
df_brg["city_hall"]     = (df_brg["Q4A"] == 1).astype("uint8")
df_brg["market"]        = (df_brg["Q4E"] == 1).astype("uint8")
df_brg["elem_sch"]      = (df_brg["Q4F"] == 1).astype("uint8")
df_brg["hs_sch"]        = (df_brg["Q4G"] == 1).astype("uint8")
df_brg["health"]        = (df_brg["Q4K"] == 1).astype("uint8")
df_brg["hospital"]      = (df_brg["Q4J"] == 1).astype("uint8")
df_brg["plaza"]         = (df_brg["Q4C"] == 1).astype("uint8")
df_brg["port"]          = (df_brg["Q4M"] == 1).astype("uint8")
df_brg["library"]       = (df_brg["Q4I"] == 1).astype("uint8")

df_brg["waterworks_system"] = (df_brg["Q4N"] == 1).astype("uint8")
df_brg["cell_signal"]       = (df_brg["Q4Q"] == 1).astype("uint8")

df_brg["landline"]      = (df_brg["Q4P"] == 1).astype("uint8")
df_brg["post_office"]   = (df_brg["Q4O"] == 1).astype("uint8")
df_brg["fire_station"]  = (df_brg["Q4L"] == 1).astype("uint8")

# -----------------------------------------------------------
# 4.  Keep a tidy list of the new columns for later use
# -----------------------------------------------------------
pmt16_cols = [
    "poblacion",
    "street_pattern",
    "acc_nat_hwy",
    "cemetery",
    "city_hall",
    "market",
    "elem_sch",
    "hs_sch",
    "health",
    "hospital",
    "plaza",
    "port",
    "library",
    "waterworks_system",
    "cell_signal",
    "landline",
    "post_office",
    "fire_station",
]


df_brg[pmt16_cols] = df_brg[pmt16_cols].astype(int)
print(df_brg[pmt16_cols].dtypes)
print(" community indicators built:", pmt16_cols)


poblacion            int64
street_pattern       int64
acc_nat_hwy          int64
cemetery             int64
city_hall            int64
market               int64
elem_sch             int64
hs_sch               int64
health               int64
hospital             int64
plaza                int64
port                 int64
library              int64
waterworks_system    int64
cell_signal          int64
landline             int64
post_office          int64
fire_station         int64
dtype: object
 community indicators built: ['poblacion', 'street_pattern', 'acc_nat_hwy', 'cemetery', 'city_hall', 'market', 'elem_sch', 'hs_sch', 'health', 'hospital', 'plaza', 'port', 'library', 'waterworks_system', 'cell_signal', 'landline', 'post_office', 'fire_station']


In [8]:
# 1. Province-level averages
prov_comm_means = (
  df_brg
    .groupby("PRV", as_index=False)[pmt16_cols]
    .mean()            # simple arithmetic mean
    .round(3)          # optional: nicer display, keep 3 decimals
)

# 2. Preview
# prov_comm_means




In [10]:
prov_comm_means.to_csv("output/pmt_comm_indicators_by_province.csv", index=False)